In [20]:
import pandas as pd 
import sys
import random
from sklearn.naive_bayes import MultinomialNB
import classifier_tool as tool

In [21]:
def get_data_file(bencana):
    #d_anotated = f'D:\\data-gusman\\tmp\\anotated_data\\{bencana}\\v02-new\\'
    d_anotated = f'C:\\Users\\dharmapu\\Documents\\personal\\ui\\KA-AMSD_src\\paper-submission\\anotated_data\\{bencana}\\v02-new\\'
    f_data = d_anotated + f'sample_{bencana}_data.json'
    return f_data

In [22]:
f_data = get_data_file('tsunami')
#f_data, os.path.exists(f_data)
#%run -i analysis_repr.py $f_data

(314, 851, 1165)

In [29]:
def data_distribution(bencana):
    f_data = get_data_file(bencana)
    df_data = pd.read_json(f_data)
    print(bencana + "\t" + str(len(df_data[df_data['label'] == 'Y'])) + "\t" + str(len(df_data[df_data['label'] == 'N'])) + "\t" + str(len(df_data)))

In [30]:
daftar_bencana = [ 'angin_topan', 'banjir', 'erupsi', 'gempa', 'karhutla', 'kekeringan', 'longsor', 'tsunami']
for bencana in daftar_bencana:
    data_distribution(bencana)

angin_topan	237	973	1210
banjir	320	924	1244
erupsi	147	943	1090
gempa	255	962	1217
karhutla	394	755	1149
kekeringan	232	866	1098
longsor	313	907	1220
tsunami	314	851	1165
